In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# Mở Chrome và cho full màn hình
browser = webdriver.Chrome(executable_path='C:/Users/Admin/Desktop/chromedriver')
browser.maximize_window()

In [ ]:
# Mở trang web cần thu thập thông tin
browser.get('https://www.truecar.com/used-cars-for-sale/listings/?sort[]=best_match')

In [ ]:
# Cột bên trái chọn 3 hãng Honda, Ford, Huyndai, Years: 2013-2020 
#- VD: năm 2013: lấy thông tin 100 xe Honda, 100 xe Ford, 100 xe Huyndai
#- Tiếp tục làm vậy đến năm 2020
#--> có tổng cộng (100*3)*8 = 2400 thông tin xe
makes = ['Honda', 'Ford', 'Huyndai']
start_year = 2013
end_year = 2020

In [ ]:
#Chọn năm kết thúc
#year_max = browser.find_elements_by_css_selector('[aria-label="Year Max"] option')
#for year_1 in year_max:
#    if str(start_year) in year_1.text:
#        year_1.click()
#        break

In [ ]:
# chọn năm bắt đầu
#year_min = browser.find_elements_by_css_selector('[aria-label="Year Min"] option')
#for year in year_min:
#    if str(start_year) in year.text:
#        year.click()
#        break

In [ ]:
#Chọn hãng (Make)
#make = browser.find_elements_by_css_selector('[aria-label="Make"] option')
#for i in make:
#    if 'Honda' in i.text:
#        i.click()
#        break

In [ ]:
#browser.quit()

In [ ]:
# Ấn nút sang trang tiếp theo
#browser.find_element_by_css_selector('[data-qa="Pagination-directional-next"]').click()

In [ ]:
#Test thu thập dữ liệu 1 o to
#link = 'https://www.truecar.com/used-cars-for-sale/listing/3N1AB7AP0KL625381/2019-nissan-sentra/'
#text = requests.get(link).text
#cay = BeautifulSoup(text, 'html.parser')
#Brand = cay.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string[5:]
#print(Brand)
##Model
#Vehicle_Overview = cay.find_all('div',{'class':"_1l9na3c align-self-center col"})
#try:
#    Body_style = Vehicle_Overview[1].text[5:]
#except:
#    Body_style = ''
#print(Body_style)
#
#Year = year
#print(Year)
#
#try:
#    Mileage = Vehicle_Overview[8].text[7:]
#except:
#    Mileage = ''
#print(Mileage)
#
#try:
#    Engine = Vehicle_Overview[4].text[6:]
#except:
#    Engine =''
#print(Engine)
#
#try:
#    Fuel_type = Vehicle_Overview[7].text[9:]
#except:
#    Fuel_type = ''
#print(Fuel_type)
#
#try:
#    Number_of_owners = cay.find_all('li',{'class':"_h9wfdq"})[3].text
#except:
#    Number_of_owners =''
#print(Number_of_owners)
#
#try:
#    Price = cay.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
#except:
#    Price = ''
#print(Price)

In [ ]:
web = 'http://truecar.com'
file = open('raw_data.csv', 'w', encoding='utf-8')
file.write(f'brand\tbody style\tyear\tmileage\tengine\tfuel type\tnumber of owners\tprice\n')
for make in makes:
    #Chọn hãng (Make)
    make_option = browser.find_elements_by_css_selector('[aria-label="Make"] option')
    for make_1 in make_option:
        if make in make_1.text:
            make_1.click()
            break
    time.sleep(5)
    for year in range(start_year,end_year+1):
        dem = 0
        #Chọn năm kết thúc
        year_max = browser.find_elements_by_css_selector('[aria-label="Year Max"] option')
        for year_1 in year_max:
            if str(year) in year_1.text:
                year_1.click()
                break
        time.sleep(5)
        #chọn năm bắt đầu
        year_min = browser.find_elements_by_css_selector('[aria-label="Year Min"] option')
        for year_2 in year_min:
            if str(year) in year_2.text:
                year_2.click()
                break
        time.sleep(5)
        html_text = browser.page_source
        tree = BeautifulSoup(html_text, 'html.parser')
        cars = tree.find_all('div', {'data-qa': 'Listings'})
        for car in cars:
            dem=dem+1
            if(dem>100):
                break
            url = car.find('a', {'class':'card card-1 card-shadow card-shadow-hover vehicle-card _1qd1muk'})['href']
            car_html_text = requests.get(web+url).text
            car_tree = BeautifulSoup(car_html_text, 'html.parser')
            try:
                Brand = car_tree.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string[5:]
            except:
                Brand = ''
            Vehicle_Overview = car_tree.find_all('div',{'class':"_1l9na3c align-self-center col"})
            try:
                Body_style = Vehicle_Overview[1].text[5:]
            except:
                Body_style = ''

            Year = year

            try:
                Mileage = Vehicle_Overview[8].text[7:]
            except:
                Mileage = ''

            try:
                Engine = Vehicle_Overview[4].text[6:]
            except:
                Engine =''

            try:
                Fuel_type = Vehicle_Overview[7].text[9:]
            except:
                Fuel_type = ''

            try:
                Number_of_owners = cay.find_all('li',{'class':"_h9wfdq"})[3].text
            except:
                Number_of_owners =''

            try:
                Price = cay.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
            except:
                Price = ''
            file.write(f'{Brand}\t{Body_style}\t{Year}\t{Mileage}\t{Engine}\t{Fuel_type}\t{Number_of_owners}\t{Price}\n')
        while True:
            if(dem>100):
                break
            # Ấn nút sang trang tiếp theo
            browser.find_element_by_css_selector('[data-qa="Pagination-directional-next"]').click()
            html_text = browser.page_source
            tree = BeautifulSoup(html_text, 'html.parser')
            cars = tree.find_all('div', {'data-qa': 'Listings'})
            for car in cars:
                dem=dem+1
                if(dem>100):
                    break
                url = car.find('a', {'class':'card card-1 card-shadow card-shadow-hover vehicle-card _1qd1muk'})['href']
                car_html_text = requests.get(web+url).text
                car_tree = BeautifulSoup(car_html_text, 'html.parser')
                try:
                    Brand = car_tree.find('div',{'class':'text-truncate heading-3 margin-right-2 margin-right-sm-3'}).string[5:]
                except:
                    Brand = ''
                Vehicle_Overview = car_tree.find_all('div',{'class':"_1l9na3c align-self-center col"})
                try:
                    Body_style = Vehicle_Overview[1].text[5:]
                except:
                    Body_style = ''

                Year = year

                try:
                    Mileage = Vehicle_Overview[8].text[7:]
                except:
                    Mileage = ''

                try:
                    Engine = Vehicle_Overview[4].text[6:]
                except:
                    Engine =''

                try:
                    Fuel_type = Vehicle_Overview[7].text[9:]
                except:
                    Fuel_type = ''

                try:
                    Number_of_owners = car_tree.find_all('li',{'class':"_h9wfdq"})[3].text
                except:
                    Number_of_owners =''

                try:
                    Price = car_tree.find('div',{'data-qa':"LabelBlock-text"}).text[1:]
                except:
                    Price = ''
                file.write(f'{Brand}\t{Body_style}\t{Year}\t{Mileage}\t{Engine}\t{Fuel_type}\t{Number_of_owners}\t{Price}\n')
file.close()
browser.quit()
print('Done!')